# 02: Knowledge-Grounded Agent Basics

This notebook introduces the `KnowledgeGroundedAgent` class, which wraps Gemini
with Google Search grounding into a full-featured QA agent.

## Learning Objectives

- Create and configure a `KnowledgeGroundedAgent`
- Understand the agent's system instructions
- Use the agent for single-turn Q&A
- Explore the DeepSearchQA evaluation dataset

In [1]:
# Setup: Load environment and configure rich console
from aieng.agent_evals import (
    create_console,
    display_example,
    display_info,
    display_response,
    display_success,
)
from dotenv import load_dotenv


console = create_console()
load_dotenv(verbose=True)

True

## 1. Creating a Knowledge Agent

In [2]:
from aieng.agent_evals.knowledge_agent import KnowledgeGroundedAgent
from rich.panel import Panel
from rich.table import Table


# Create the agent
agent = KnowledgeGroundedAgent()

# Display configuration
config_table = Table(title="🤖 Agent Configuration", show_header=True, header_style="bold cyan")
config_table.add_column("Setting", style="cyan")
config_table.add_column("Value", style="white")
config_table.add_row("Model", agent.model)
config_table.add_row("Planner Model", agent.config.default_planner_model)
config_table.add_row("Worker Model", agent.config.default_worker_model)

console.print(config_table)

       🤖 Agent Configuration       
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Setting       ┃ Value            ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Model         │ gemini-2.5-flash │
│ Planner Model │ gemini-2.5-pro   │
│ Worker Model  │ gemini-2.5-flash │
└───────────────┴──────────────────┘

## 2. Understanding System Instructions

The agent uses carefully crafted system instructions to guide its behavior.

In [3]:
from aieng.agent_evals.knowledge_agent.agent import SYSTEM_INSTRUCTIONS
from rich.markdown import Markdown


console.print(
    Panel(
        Markdown(SYSTEM_INSTRUCTIONS),
        title="📜 System Instructions",
        border_style="blue",
        subtitle="[dim]Guides agent behavior[/dim]",
    )
)

╭──────────────────────────────────────────── 📜 System Instructions ─────────────────────────────────────────────╮
│ You are a knowledge-grounded research assistant. Your role is to answer questions accurately by searching the   │
│ web for relevant information.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                             How to Answer Questions                                             │
│                                                                                                                 │
│  1 Search First: Always search the web before answering factual questions that require current information. Do  │
│    not rely solely on your training data.                                                                       │
│  2 Be Thorough: For complex questions, search multiple times to gather all relevant facts before synthesizing   │
│    your answer.                                                                                                 │
│  3 Cite Sources: Always mention which sources you used to answer the question.                                  │
│  4 Be Honest: If you cannot find relevant information, say so clearly.                                          │
│  5 Synthesize Information: When answering complex questions, synthesize findings from multiple sources into a   │
│    coherent response.                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│                                                 Response Format                                                 │
│                                                                                                                 │
│ When answering questions:                                                                                       │
│                                                                                                                 │
│  • Provide a clear, direct answer first                                                                         │
│  • Include relevant context and details from your sources                                                       │
│  • List the sources used at the end of your response                                                            │
╰───────────────────────────────────────────── Guides agent behavior ─────────────────────────────────────────────╯

## 3. Single-Turn Q&A

Let's use the agent to answer some questions.

In [4]:
# Example 1: Current events
question = "What are the most significant AI developments in January 2026?"

console.print(
    Panel(
        f"[bold green]{question}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

console.print("[dim]🔍 Agent is researching...[/dim]")
response = await agent.answer_async(question)

display_response(response, console=console, title="AI Developments January 2026")

# Show tool calls made during reasoning
if response.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls (ReAct Trace):[/bold cyan]")
    for tc in response.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

╭────────────────────────────────────────────────── ❓ Question ──────────────────────────────────────────────────╮
│ What are the most significant AI developments in January 2026?                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔍 Agent is researching...

╭──────────────────────────────────────── 📖 AI Developments January 2026 ────────────────────────────────────────╮
│ January 2026 has witnessed substantial advancements and key trends shaping the landscape of Artificial          │
│ Intelligence, with a strong focus on autonomous systems, specialized applications, enterprise integration, and  │
│ evolving model capabilities.                                                                                    │
│                                                                                                                 │
│ Key AI Developments in January 2026 include:                                                                    │
│                                                                                                                 │
│  • Emergence of Autonomous AI Systems: A significant trend is the shift towards autonomous AI systems capable   │
│    of independent decision-making, executing multi-step tasks, and functioning as digital collaborators.        │
│    Enterprises are moving beyond basic "copilots" to fully autonomous AI workflows across various functions     │
│    like finance, HR, customer support, and supply-chain management, leading to substantial productivity gains.  │
│  • Advancements in AI Models:                                                                                   │
│     • Leading Models: Several AI models have garnered attention, with Claude Opus 4.5 recognized for its strong │
│       reasoning and writing, GLM-4.7 Thinking as a top open-source model, and Gemini 3 Pro excelling in speed   │
│       and multimodal capabilities. GPT-5.2 is also highlighted for pushing the boundaries of raw intelligence   │
│       and complex reasoning.                                                                                    │
│     • Compact High-Performance Models: The Technology Innovation Institute (TII) announced Falcon-H1R 7B, a     │
│       compact model with only 7 billion parameters that performs comparably to systems seven times its size,    │
│       making it ideal for "edge AI" in manufacturing.                                                           │
│  • Specialized AI Applications and Breakthroughs:                                                               │
│     • Drug Discovery: NVIDIA and Eli Lilly launched a $1 billion AI co-innovation lab to transform drug         │
│       discovery and pharmaceutical manufacturing, aiming to compress timelines and improve molecular screening  │
│       accuracy through "physical AI" and advanced simulations.                                                  │
│     • Genetic Disease Identification: Google DeepMind introduced AlphaGenome, an AI tool that can analyze up to │
│       1 million letters of DNA code to predict how mutations affect biological processes, aiding in identifying │
│       genetic drivers of diseases like cancer and potentially underpinning new gene therapies.                  │
│     • Weather and Climate Prediction: NVIDIA unveiled the Earth-2 family of open models, including Earth-2      │
│       Medium Range for high-accuracy 15-day forecasts and Earth-2 Nowcasting, which uses generative AI to       │
│       predict local storms with kilometer-resolution, outperforming traditional physics-based models.           │
│     • Manufacturing Robotics: Boston Dynamics' humanoid robot, Atlas, powered by Nvidia's AI chip, began its    │
│       first field test at a Hyundai plant, autonomously performing tasks and demonstrating "motion capture      │
│       learning" capabilities. Hexagon Robotics also partnered with Microsoft to develop humanoid robots for     │
│       data-driven manufacturing.                                                                                │
│  • Enterprise Adoption and Strategic Investment: AI is transitioning from experimental phases to large-scale    │
│    implementation as a strategic necessity. Global AI s

✓ 5 queries | 16 sources

🔍 Search Queries
├── 1. significant AI developments January 2026
├── 2. major AI breakthroughs January 2026
├── 3. new AI models January 2026
├── 4. AI research highlights January 2026
└── 5. AI news January 2026

📚 Sources
├── [1] ]8;id=956789;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQG7k1VH-FZVMVoUDaslBt_aI318rqifrHbDwl4YdAv3hBPbuVyJFCElvK92s_0uACRAfqMaPCmsHTSXWJzhGQ9oQOW7iAnQ7-ZxOamAIRqnzVCm3sXvGIkZ5cq4j7AVJ7Uw__R_3pO_KgTm-8FabZ25MewFzHCnUpQ4YgxjpYzQfe5Er7BBi5P94SjjdCcJ-OSt3-GjLy0hudCPahh5EHs6nvWYRAE=\trigyn.com]8;;\
├── [2] ]8;id=640236;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEcTPkTITfa7TeoGDySQzkK3fw7MZOrd5Uhvtp25G1HoELFIfHPg-l_phZPGfuMFxhh7-eqgp7W4pxJffFouG40DV9goHu9pSHt1OBj6DSqRR4VrPJwIEcRenlgx8kuQebN73h1uDauxkBSBt32Osk=\whatllm.org]8;;\
├── [3] ]8;id=118478;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEFaSNv7RymzK2lUBITgSrOznjMlGcD0E_jbYoTC_YPMYpANu7HCZP2hqNcEef7FIKFvpIlUNfIJVfVSs2M_2unUP7WZ6Dro_UB8v1V4ymF1Q0xTS62fJdxidxox8uh6g21HEhGLlU=\felloai.com]8;;\
├── [4] ]8;id=154662;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGeyaJO1glIvb3XwlyexmJ8JdERw4jTjL665rdG7VCsen5afW3uRevRTSMFRumRzCmQrKkoZynBFdKrVo3c1qUvFkZvzTjIeHEq3QSsRLhxqhjK5vPTW3ywyUnm4RZD8CL-1GVSajaDn3Y0JXXiOHgghrItuRsk6vaMJzdNv2TujNhNk4rHudzPDtdYdlEK-vBwfPriK-OjM7D47c9_w8R5mx5n2o2bWg==\amiko.consulting]8;;\
├── [5] ]8;id=865337;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFybuYhbZH7GE0kndHZK8npApfyHD0EE0E-IEOfSF9tXWteowrGu4ly2Vp8iQYOQXknPfWrqju9BumZgww9esWoQYkXtYBkqlGWIV5gEyJXeX-0gMXJ77f6gcIBfELEG5rBPcLIubYSymtaXYB64iAmluOImJ7xvyF-TH_HFgLNskJ8INp8Goqfca5sT0pzBaRP4jqQcSa1VOgaHTZyHAE=\artificialintelligence-newstoday.com]8;;\
├── [6] ]8;id=110008;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEh2cOjT11W3AiKV9wfUAm1-FrlIpA6m7YhtDRPutYe2QpGiJ_Uw_0VazR0ALMRRriBkA97Ok-7t6p5Xky1BSFlTg5jtBcc6PedE_U3ZbA2bBKJxB-VPJ4DX_hxX7oHJVN95DQ515LHmc1vSH6Xosgk8LLOjAQNe71Bu8nK61TVNNSJiZish7BZKEQPICDJbLwhabOBpYXZV1YE__Cjqg==\theguardian.com]8;;\
├── [7] ]8;id=707717;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEbmneYJS6wqSpioIzcLWwVpciTuX0fstsMUVkfq_cTQ9BsUsEKcaCIxK8LX_9xnVoKsfSCCJDl92gJe2zD7yvkNuJYLESSUGuJQ0AoKo6QobvIpRHTROCZp9jlEhSA-99-17Lhe2KRKaeW-NRJrZ9ucf_6\nvidia.com]8;;\
├── [8] ]8;id=365176;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFvhwvoT-N16KsFk6BWPXHvilYExcOy8tC8_He87Pl1Yjk8h1uUSxmLjG3Owd9Z7W_q1a9BoQFO1J_vHL7tB7IV0tzTZDdpTDrQvuCuh_heEUk1Kq4WmJFiV_NH-V3BBn1tlTlC2JkYoyAU2PAOJ5oTKsPn45BQkbaZ4oPE4Ihhj6n9shF1Ezbf0u9Qttc=\cgspam.org]8;;\
├── [9] ]8;id=102687;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH9uP5ER91H3By3wi4xA8pz3c_CaSbaX_yIb4vwqtw0Na9VYXjTbPAcCMoSY3UcQCoNQgoHTyEiAeQ-hfSe-ccKNC-ydnfdQfRNgOr4vaycclajGBsQkcy-Vl3j6AxIht8IwKQ-8SC3OawZ9itVU15Dp6QVjQyGJhf4LWEGR_84C69tMcqe5OMaca_r3tHbOOodxnViov5890ZIqFz8R6fFaoA=\aljazeera.com]8;;\
├── [10] ]8;id=479461;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGcTYnVIivNptsHC1HFFYFNgwIkKiHVoXoxSmqy3SIQNAeKABAWLJU9DTEiu3KXUtr5hE99ddIXGRIx26bi7XCHgDV08LRS18BjFWXZK0fQ_xk-oKckWsdBv0Ws1n_xRHxpggeWokcoCoP-FkOw3zsbSd_ckI052b3rphuZTgK6_AtHbh8FIOR5Mvana_79Hv8QqGd9Pc6Bwg==\business20channel.tv]8;;\
├── [11] ]8;id=37652;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGf9qcIP1PMlnE0qmhP2IqRVh-vUjgM61flqaC6k-Xg0n91du7egPl-f6OtCY7NXLGsVAnEA_gV0538EUX6n2g2XLPGzq55yMoucM3CnaSmr-tMlbRM4RvU6kVrerywZa3urf44U44yQo_6KmZ_ooPJclQoHHWkdKtLoSf4FdiNTWJAAqhvNBovCf1WqA==\sap.com]8;;\
├── [12] ]8;id=449910;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGjl38ACvnoboSnEpkb8PFaXqM6J4UOEQVKpYXHKetCBTY8x3op_7jMU3-egpuZT_pe2xgEnJojfAIdnc8Hb9s0nL87_Dkg1tJheU48gjJV2Q73r-ZFAQHrQuFanC4NHstdlpqDBjSxatYNaNJI2j_ikaOsK2llG1KsC1A6jNuhDU0KDzBInVC9DFJIHVA2LB1f5-IUt2yBho1YXQX-o_U7angHX2HV\theguardian.com]8;;\
├── [13] ]8;id=773605;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQF7Ftzs7rKlLjTYoMwgAu5GsBXRXKJN8l8x6hob4KjE97wKUO0C0srn21iHRnEpSAED7e3O3_Yo9GBEbPgYgDXKhALmCQGcyLXhNSH8e_f-MQGYsk1lM8HRryzQxk4lkl5US56CM8mCnoDPLgOehiy3sA==\nga.org]8

In [5]:
# Example 2: Factual question
question = "What countries have successfully landed spacecraft on the Moon?"

console.print(
    Panel(
        f"[bold green]{question}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

console.print("[dim]🔍 Agent is researching...[/dim]")
response = await agent.answer_async(question)

display_response(response, console=console, title="Moon Landing Countries")

# Show tool calls
if response.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls:[/bold cyan]")
    for tc in response.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

╭────────────────────────────────────────────────── ❓ Question ──────────────────────────────────────────────────╮
│ What countries have successfully landed spacecraft on the Moon?                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔍 Agent is researching...

╭─────────────────────────────────────────── 📖 Moon Landing Countries ───────────────────────────────────────────╮
│ Five countries have successfully landed spacecraft on the Moon: the Soviet Union (now Russia), the United       │
│ States, China, India, and Japan.                                                                                │
│                                                                                                                 │
│ Key achievements in lunar landings include:                                                                     │
│                                                                                                                 │
│  • Soviet Union (Russia): The Soviet Union was the first to achieve a successful soft landing on the lunar      │
│    surface with Luna 9 in 1966. Their Luna 2 was also the first human-made object to impact the Moon in 1959.   │
│  • United States: The United States achieved its first soft landing with Surveyor 1 in 1966. The U.S. is also   │
│    the only country to have successfully landed humans on the Moon, with the Apollo 11 mission in July 1969     │
│    being the first, followed by five more crewed landings through 1972.                                         │
│  • China: China's Chang'e-3 mission achieved a soft landing in December 2013. In January 2019, China's          │
│    Chang'e-4 made the first-ever soft landing on the far side of the Moon.                                      │
│  • India: India became the fourth country to achieve a soft landing on the Moon with its Chandrayaan-3 mission  │
│    in August 2023, also becoming the first to land near the lunar south pole.                                   │
│  • Japan: Japan successfully soft-landed its Smart Lander for Investigating Moon (SLIM) mission in January      │
│    2024, making it the fifth country to achieve this feat.                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ 2 queries | 6 sources

🔍 Search Queries
├── 1. countries that have successfully landed spacecraft on the Moon
└── 2. nations with successful lunar landings

📚 Sources
├── [1] ]8;id=112120;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGhA6uKmAIVHmC_L4JL58vYa1gFmkHkLD4C1so5C09POGTHZDajQnsGMGg_2ulGh-cEoBvx_hjrrfHp0QBeUr38aBZcSgntY1mgCJcJK8AccLMbJZhLBZSMqEjkJnDyIfhudQIM\wikipedia.org]8;;\
├── [2] ]8;id=291649;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGvwApIrTKxTWF6qJkfAbP9UBCtAzm_8XsZ4aDrjx17_RM_02P7O9C59PieoGQY22Lyq-9UjJEQdeYs2157p5LBuSw8cKjptSY0Rk58rXWollS40DloJIZQ89vd6XnSpvtUdR83C_Pp_GAEuTFZ2n0Dbf0meQ==\starlust.org]8;;\
├── [3] ]8;id=160046;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGo3wJtvtuG7Ko9ywNap-SXsgZ2HNduQfgv6cmIjgtV3ISkmoaoo-UOuYxzLPi6rp3kqpsPjObpob6dXgnBzuTV9KoUiXzbom16NQqVG7iQCg0qk7y33C3o_rvv25FrD1dxFjsl_A==\youtube.com]8;;\
├── [4] ]8;id=546875;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGOYHuq63obBdBQUIHbTenzAbR5r8Uq4h6JoAjqrUJvY8TWzevUFPHrqNyXZF5vCXUFeENigmxCECFE63gPIAA7EFxwtfhgILD9dKOAH_RO9DW5LVfKfIzDhhCfk9FAVpmgYMl-IhS5reypyUa7yVpzBedRB___IirHaYFdI7GSqHT0tU03JvQzP2Q_\tbsnews.net]8;;\
├── [5] ]8;id=194902;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEUOLyi_xfio5pKJww34v5Y9HC8jXeoABTFHp3RI2SGGV0VdbxqfvCTkh1S4TKOx1eeKcsJxyh2tTmtsNUrT6uJ4rQle56q-I_6L4yYMWkQ1F6XMhpSvzakr4FgqtwXUhrX6F05roRfC2Y-WLh6dGptnAHN7w==\wikipedia.org]8;;\
└── [6] ]8;id=134044;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFyaO196mks9_J-mIa7peNBn0gQ-hW6Yo2FMWGJZonuOQMml40-OvKJX_rAG58P-tx_Ea9INU6cWXRfI4hl6cO1zNKAlpW1z166N0G9V9OyIb2yng9KvZ5qwYmlOk1o4lUpsu7FYiG2AhRrQyI7ckWlmRSKB6I=\planetary.org]8;;\

## 4. Using the AsyncClientManager

For applications that need to manage client lifecycle, use `AsyncClientManager`.

In [6]:
from aieng.agent_evals.knowledge_agent import AsyncClientManager


# Create a manager (lazy initialization)
manager = AsyncClientManager()

display_info(f"Initialized: {manager.is_initialized()}", console=console)

# Access the agent (triggers initialization)
managed_agent = manager.agent
display_info(f"After access: {manager.is_initialized()}", console=console)

# Use the agent
console.print("[dim]Querying...[/dim]")
response = await managed_agent.answer_async("What is the speed of light?")

display_response(response, console=console, title="Quick Answer", show_queries=False)

# Cleanup
manager.close()
display_success("Manager closed", console=console)

ℹ Initialized: False

ℹ After access: True

Querying...

╭──────────────────────────────────────────────── 📖 Quick Answer ────────────────────────────────────────────────╮
│ The speed of light in a vacuum, often denoted as 'c', is a universal physical constant. It is exactly           │
│ 299,792,458 meters per second (m/s).                                                                            │
│                                                                                                                 │
│ To put this into perspective:                                                                                   │
│                                                                                                                 │
│  • It is approximately 1.07 billion kilometers per hour (km/h).                                                 │
│  • It is about 670,616,629 miles per hour (mph).                                                                │
│  • If you could travel at the speed of light, you could circumnavigate the globe approximately seven and a half │
│    times in one second.                                                                                         │
│  • Light from the Sun takes about 8 minutes and 10 to 27 seconds to reach Earth, depending on the time of year. │
│                                                                                                                 │
│ The speed of light is a fixed value because the meter is defined by international agreement as the distance     │
│ light travels in a vacuum during a specific time interval (1/299,792,458 of a second). It is a constant for all │
│ observers, regardless of their relative velocity, and represents the upper limit for the speed at which         │
│ information, matter, or energy can travel through space. While light travels fastest in a vacuum, it slows down │
│ when passing through other materials like water or glass.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ 2 queries | 7 sources

📚 Sources
├── [1] ]8;id=963545;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4YBRb5BqY1UQy4QLl8SiKVefU5e3ou7m-NgV6JLro30m-ttMNng6os7ZOaktwqdYyfkH9rzEkVIiMOB1T2xya8RtnrWzmhsjJQ5UKsjgqQvR55CnG0d0Orf4NkuJctbkRQmiNHd4=\wikipedia.org]8;;\
├── [2] ]8;id=209123;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGtBVwlSIDTmS1OtW8VQvc1vhokv7RH3bOo0thXSbhigSrpnZh-PkI_-8CN3di4Hb4p6G9LtCbb2YAYieA8cj8HP9QRWsD3n7GDMoc-GkFcZM-d-o4NmbJg7_JuQXev1Via4NTTjlN8CEYhi-CpX7gGdz-f0qlLLidIaCziXa5HvfmsxPbGAGnaNzI=\astronomy.com]8;;\
├── [3] ]8;id=483821;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHg8DszPKWNoIJYOyTLDK7vwOYnqvQnSCl8M4I7GvuFueiciHWGz4_OI5RYeOgmTAAu_nfmPfCIkdLY6Z7JJO6VLs4Tkrpi5YFEFQOYG3BQ3S2vOlWagIBaa18iVR8ykevB5aY8gEapgo3SPwm1LAix8A==\universetoday.com]8;;\
├── [4] ]8;id=574180;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE3gOMPm0dHcooD1S54p2o8esnPcoi8ByTCP1geJDOgisyhU2n136wCL3ocezWPatKuMMkjdZSm4JEocAJ87EjZwx2dIxN0kmUytIb4nODZlBTq3KV3ASgokF1IYz52ls0yiZJF\youtube.com]8;;\
├── [5] ]8;id=955351;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH4-tKvR_yI4mJsbVe-NpSp7bOz3Tr6JIbNm2xIemyXGe_JYvbGB9LSdW9SAG-vLFeIcCVSIByklc5Lgi5qRAVSgx023IuSRkY6jUUNgW14kJPExIkGj8_B63gg5CZ_WYhzMIjgQA==\youtube.com]8;;\
├── [6] ]8;id=974046;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFCGVRVNTveBngyLk4EvAtZZZ5kTPoiYKS29-TWMW5J8KY5VWcHa_q9meTh6Fy07j4riG9_OPFdnkWfja1v1CU4vCJLnkPOMYZvD_L9RgB-0HW2hVcySopAEwnzypFSmD8C02w5VLQWszY=\lco.global]8;;\
└── [7] ]8;id=683796;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFOwx8lfR3JUYfMO0RIbCPcXbJmdVFuITEXqBt4pZmFtvbYLeap9HeInN3sPP8Zg3ncWahCTBQwCooiYq3tPG1356O8yeGj-wRjVoAW1Veh-ZoTqJDcjCkvEOU0MFCwdl1Fv7Ea2BBOu7q4tCvyLK1K8ozSVmMkDrmWqVlsmgl5Mkv-p_G6K1vruLjjLU_LIEQaIeeDBGNr\evidentscientific.com]8;;\

✓ Manager closed

## 5. Exploring the DeepSearchQA Dataset

The DeepSearchQA dataset contains 896 research questions for evaluating knowledge agents.

In [7]:
from aieng.agent_evals.knowledge_agent import DeepSearchQADataset


# Load the dataset
with console.status("[cyan]Loading DeepSearchQA dataset...[/cyan]", spinner="dots"):
    dataset = DeepSearchQADataset()

# Display dataset info
info_table = Table(title="📊 DeepSearchQA Dataset", show_header=True, header_style="bold cyan")
info_table.add_column("Metric", style="cyan")
info_table.add_column("Value", style="white")
info_table.add_row("Total Examples", str(len(dataset)))
info_table.add_row("Categories", str(len(dataset.get_categories())))

console.print(info_table)

 📊 DeepSearchQA Dataset  
┏━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric         ┃ Value ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Total Examples │ 896   │
│ Categories     │ 17    │
└────────────────┴───────┘

In [8]:
# Display categories
categories = dataset.get_categories()

cat_table = Table(title="📁 Problem Categories", show_header=True, header_style="bold green")
cat_table.add_column("Category", style="white")
cat_table.add_column("Count", style="cyan", justify="right")

for cat in sorted(categories):
    count = len(dataset.get_by_category(cat))
    cat_table.add_row(cat, str(count))

console.print(cat_table)

      📁 Problem Categories      
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Category              ┃ Count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Arts                  │    26 │
│ Arts & Entertainment  │     1 │
│ Biology               │     2 │
│ Current Events        │     3 │
│ Education             │    94 │
│ Finance & Economics   │   132 │
│ Geography             │    95 │
│ Health                │    92 │
│ History               │    44 │
│ Linguistics           │     1 │
│ Media & Entertainment │    27 │
│ Other                 │    65 │
│ Politics & Government │   148 │
│ Science               │    89 │
│ Sports                │    19 │
│ Technology            │    22 │
│ Travel                │    36 │
└───────────────────────┴───────┘

In [9]:
# Show a sample example using the shared display utility
example = dataset[0]

display_example(
    example_id=example.example_id,
    problem=example.problem,
    category=example.problem_category,
    answer=example.answer,
    answer_type=example.answer_type,
    console=console,
)

╭───────────────────────────────────────────────── 📝 Example 0 ──────────────────────────────────────────────────╮
│ Problem:                                                                                                        │
│ Consider the OECD countries whose total population was composed of at least 20% of foreign-born populations as  │
│ of 2023 (according to the Observatory of Migration at the university of Oxford). Amongst them, which country    │
│ saw their overall criminality score increase by at least +0.2 point between 2021 and 2023 and their resilience  │
│ score decrease by more than 0.3 between these same dates (according to the Organised Crime Index)?              │
│                                                                                                                 │
│ Category: Politics & Government                                                                                 │
│ Answer Type: Single Answer                                                                                      │
│                                                                                                                 │
│ Expected Answer:                                                                                                │
│ New Zealand                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
# Get random samples
samples = dataset.sample(n=3, random_state=42)

console.print("[bold]📚 Random Samples from Dataset[/bold]\n")

for ex in samples:
    display_example(
        example_id=ex.example_id,
        problem=ex.problem[:300] + "..." if len(ex.problem) > 300 else ex.problem,
        category=ex.problem_category,
        answer=ex.answer,
        console=console,
    )

📚 Random Samples from Dataset

╭──────────────────────────────────────────────── 📝 Example 714 ─────────────────────────────────────────────────╮
│ Problem:                                                                                                        │
│ Identify all the federally-recognized Indian Tribes in Washington State that operate tribal casinos in Skagit,  │
│ Snohomish, King, or Pierce county, and had their original gaming compact signed by Governor Booth Gardner in    │
│ 1992.                                                                                                           │
│                                                                                                                 │
│ Category: Politics & Government                                                                                 │
│                                                                                                                 │
│ Expected Answer:                                                                                                │
│ Swinomish Indian Tribal Community, Upper Skagit Indian Tribe                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Example 442 ─────────────────────────────────────────────────╮
│ Problem:                                                                                                        │
│ Of the species that were added to the Endangered Species List the same year the Endangered Species Act          │
│ celebrated its 10 year anniversary, which of those species, no matter their current ESA status, have their lead │
│ region listed as the Southeast Region? List them by their scientific names.                                     │
│                                                                                                                 │
│ Category: Science                                                                                               │
│                                                                                                                 │
│ Expected Answer:                                                                                                │
│ Palaemonias ganteri, Peromyscus gossypinus allapaticola, Neotoma floridana smalli                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📝 Example 527 ─────────────────────────────────────────────────╮
│ Problem:                                                                                                        │
│ I'm travelling on the South Western Railway service from Shepperton to London Waterloo via Kingston, which is   │
│ the first station on my route that has both ticket gates and step-free access for all platforms?                │
│                                                                                                                 │
│ Category: Travel                                                                                                │
│                                                                                                                 │
│ Expected Answer:                                                                                                │
│ Kingston                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 6. Testing the Agent on DeepSearchQA

Let's test the agent on a sample question from the dataset.

In [11]:
# Pick an example
test_example = samples[0]

console.print(
    Panel(
        f"[bold]Testing on Example {test_example.example_id}[/bold]\n\n"
        f"[cyan]Category:[/cyan] {test_example.problem_category}\n"
        f"[cyan]Expected Answer:[/cyan] {test_example.answer}",
        title="🧪 Test Setup",
        border_style="yellow",
    )
)

# Ask the agent
console.print(
    Panel(
        f"[bold green]{test_example.problem}[/bold green]",
        title="❓ Question",
        border_style="green",
    )
)

console.print("[dim]🔍 Agent is researching...[/dim]")
response = await agent.answer_async(test_example.problem)

display_response(response, console=console, title="Agent Response")

# Show tool calls
if response.tool_calls:
    console.print("\n[bold cyan]🔧 Tool Calls:[/bold cyan]")
    for tc in response.tool_calls:
        console.print(f"  • {tc.get('name', 'unknown')}: {tc.get('args', {})}")

# Compare
contains_answer = test_example.answer.lower() in response.text.lower()
if contains_answer:
    console.print("\n[green]✓ CONTAINS EXPECTED ANSWER[/green]")
else:
    console.print("\n[yellow]⚠ Answer may differ[/yellow]")
console.print(f"[dim]Expected: {test_example.answer}[/dim]")

╭───────────────────────────────────────────────── 🧪 Test Setup ─────────────────────────────────────────────────╮
│ Testing on Example 714                                                                                          │
│                                                                                                                 │
│ Category: Politics & Government                                                                                 │
│ Expected Answer: Swinomish Indian Tribal Community, Upper Skagit Indian Tribe                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── ❓ Question ──────────────────────────────────────────────────╮
│ Identify all the federally-recognized Indian Tribes in Washington State that operate tribal casinos in Skagit,  │
│ Snohomish, King, or Pierce county, and had their original gaming compact signed by Governor Booth Gardner in    │
│ 1992.                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🔍 Agent is researching...

╭─────────────────────────────────────────────── 📖 Agent Response ───────────────────────────────────────────────╮
│ The federally-recognized Indian Tribes in Washington State that operate tribal casinos in Skagit, Snohomish,    │
│ King, or Pierce county, and had their original gaming compact signed by Governor Booth Gardner in 1992 are:     │
│                                                                                                                 │
│  • Swinomish Indian Tribal Community (Skagit County). Their original gaming compact was signed by Governor      │
│    Gardner on December 21, 1992. The Swinomish Indian Tribal Community operates the Swinomish Casino & Lodge.   │
│  • Upper Skagit Indian Tribe (Skagit County). Governor Gardner signed their original gaming compact on December │
│    21, 1992. The Upper Skagit Indian Tribe operates The Skagit Casino Resort (also known as Skagit Valley       │
│    Casino Resort).                                                                                              │
│                                                                                                                 │
│ Other tribes operating casinos in the specified counties had their original gaming compacts signed in different │
│ years or by different governors:                                                                                │
│                                                                                                                 │
│  • The Tulalip Tribes (Snohomish County), which operate the Tulalip Resort Casino and Quil Ceda Creek Casino,   │
│    had their original gaming compact signed by Governor Gardner on August 2, 1991.                              │
│  • The Muckleshoot Indian Tribe (King County), operating the Muckleshoot Casino Resort, entered into their      │
│    original Class III gaming compact on February 19, 1993.                                                      │
│  • The Puyallup Tribe of Indians (Pierce County), operating Emerald Queen Casino and BJ's Bingo & Gaming, had   │
│    their original gaming compact signed by Governor Lowry on May 28, 1996.                                      │
│  • The Snoqualmie Tribe (King County), operating the Snoqualmie Casino & Hotel, had their original gaming       │
│    compact signed by Governor Locke on August 22, 2002.                                                         │
│  • The Stillaguamish Tribe of Indians (Snohomish County), operating Angel of the Winds Casino Resort, had their │
│    original gaming compact signed by Governor Locke on December 11, 2000.                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✓ 16 queries | 15 sources

🔍 Search Queries
├── 1. tribal casinos Skagit County Washington
├── 2. tribal casinos Snohomish County Washington
├── 3. tribal casinos King County Washington
├── 4. tribal casinos Pierce County Washington
├── 5. Washington State tribal gaming compacts 1992 Governor Booth Gardner
└── ... and 11 more

📚 Sources
├── [1] ]8;id=769905;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFAx2VGST7dlqUD406M1OSbwvQURbHMgvdANTAAb6Apa_ROZxJ0-KvxF71s7bmc5LWdlTfWnFWRPYv9BnJb71kjOCVn5WL-bAQLbirtBIjBLvRtRqNhc8bZQFEdl_fIucBunc3PUXj1leEfRmPPNn6bWPkECLFoN1GlE1lOCsI-JfRotgUaXVm7SHnZGLSCnrvpoCnsH_V0_7xHpBe6b8vZlPWFBl0FjIhKisn0EVKY7HcJFJ4TXolAa5jKxlVh4X0H5GB_1z_tOifffj5GbrOniYCXhbA0oi_UD-jOBXeC\bia.gov]8;;\
├── [2] ]8;id=479944;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFefz4Lm2_ciKovhSnMXA7LGP2_Hc6l5-apYVLAI6vXaf7hkc-LD4nZ_Mg3EAcPvIMu3nhbdfgGwu5fky0E4w94f3HhW6tWdZKdL2tx3VfzPcGbBde3k1FJgg59gg3Kksk9JFKxOa4GMDRbo8oEEO5eJxguArwvQgrdLqFKxIWE2VnB5PV7OCHDaeUo2yzmKgjN9QSrgU0jEtuK5hxV2bJeOhQ3z3ngjmlGO0KYog==\wa.gov]8;;\
├── [3] ]8;id=232487;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFrxH-tOrZgs49FGvfP-l-ZrWRPUH6Rixm9P2IprycQ74OerHjcJQEdwxWIqGiu_uymPdwcsyeRsqX7bs1rn8W7QJ0TCC6E8NNE2GPrljjIPMawIlMp8MbbDhhxiEMwfhGF2m5jYtXlrMiraedDLFT1Yw1iIBcWmt6ZqfKVYA3aQp-sbOiimC8B8ScUXoTJBXgdBgh0NwkEiZSpUtvMbxML-FtD3xlDOl9XLkW_ZxwDr-8yveM5V4jfaOYsYCCm4IPEKwhQSZVr8q-EsncwvQ==\bia.gov]8;;\
├── [4] ]8;id=974356;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFO1ozzaJ8SVpspGPUMUn7cqGEv5Zz2Hf81tvxaSYRMrf4tNnF4ynIUkp9-q_xu2PME37Xpx1sASxVGEydO2GMwr4n2bVouNFz4RwWDnWfzeJ00G8VF3ni2G7Xo0wpGrZqkGyQsnvvIam3NQAGjBqYc5m2EcSYR0v6VYPk-laIadbiUbv0WZF09JY5OHxKXD-iCsTdpdWBS8lLMke5yrGNc2Twu7-B_R54Qk1BQTGWBp93e0Q5Nm7AnOFVmnvTIqq8gXA==\wa.gov]8;;\
├── [5] ]8;id=976031;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH5VOrKZBHSTQke4jugy73gNrHNJKNs_N0KcOcv2oEFIWtspo_xU9a0zEY57IxTMFOZKBQQue2WwkRoJxHIC5EgFJYc-NeJgQCApQoEdi9RIkWzJW5qSNe9vMGIE81BHuO1bTOduq_tw2mv1_H3pN_6zpP7b7ZM89Q4UEuItp51204MNjR3Mc5WguZTYVkd44llHuRdOe2wqYpdVPnITKt_3WEK1dZ3wN7KF_hZtBRmyE4mgISzc9-vzBSFlzVMwrvd75mz022B9fs7gAvv9lI=\bia.gov]8;;\
├── [6] ]8;id=802788;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEqn56kUqid7fvE8i7WxcpI7hQJFnFqqip8r8pIbJdRHFdmjloYKwkCZGi4ARDe_g0n_fgW7zXbWZElgBCskkMEsD9FtiqXGcP7d-QzlpLD4vc2U144sDPOFjAdAICAJ3U3e9BWutj602jyf-ZEBzD_O7aQQ0w6luTRr5-Jk-uj_V0lXolir94=\wa.gov]8;;\
├── [7] ]8;id=995590;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHkh5NO9clX74r3D14C_EqBbwTv7KK7UNp2ux9FPO3trFA8B1KQ0Y7md4mdvaq8Zi-RGdsEqr8eoU8C7hg_REozRbEj3_EA4ZXiqkT7TGgJtEQtN_V50XEkjhx4clOgxqQG3Bj8p7XJvMMepJkT1AcOxpTPQQa_q7kElaX8knOkvc8_MUW3iCxZzDlFyWlyD8yTmu7CUc_nUmzXgv8VM8T9aCBwzEth_kTc99u-lWfS51jH8EE8kcM=\wa.gov]8;;\
├── [8] ]8;id=346893;https://www.google.com/search?q=time+in+Chelan+County,+US\Current time information in Chelan County, US.]8;;\
├── [9] ]8;id=741792;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHNUL9iOttRe1sRLaxeo3sWqRosawRCEMPpbO2c833u_YjNjO5NXMORp9KLpP8ODbmnEgRcIuW-YXVAg4SMCQJvkwCyzIp-EfW2TNo-JRooixiReocEKf95el7DoTXmxc8ZpTV6ihDpVsvAHWZP8xHYrh5K0PtDkskAv9LJ3ahh0oaOVseAO_NijhK1veXCKtA8yg9sNQbyWd2y2mTcike2JdVxu_nlkDBCvmjjONbA99euy4KM2mctYp1u3yW4ck2cDk65SUGOopBpJlnn6PuykkML\bia.gov]8;;\
├── [10] ]8;id=300807;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQE2hcWy5MaBIyS1HDgHTGS900clLhtCtxpZWGv_RK6DYdEbZ6O30RQsft1Pc_Ou1Y4Pnuvl4xdLRtXIxxHH5OHkBU1xHEGVnApTxImlqXxpvBZaLdpzki0tW_vyqG8bD_FpHgyDTHenhescA-GjBN1512QTwQKM6nJJrjEXtUjutrf8_5xSpwE4_KHS2ShS1eAQgh9E1ooJBuecy_UhgMVA6QgG6UphDWGpuV_4X80jffAhgJ5OmQ==\wa.gov]8;;\
├── [11] ]8;id=703346;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHYnKwjMm7Ug9a6TJ_xPV2wkbJKzS-j76jCH4D1nS6nf0GM1yIFdrReQuYF3KgAeiVpmEdjZbBei2Q2hltEzMPSVgaXFYzGDd78I5perdnWkJ9ECpj4xDUmf4YNBxtBj50YoWKR7Br7IUaKpcmkzZwXqdcqVBlys1j0SHHuZ6QlS_UwGhEx4bVqMPEhpDkFBR1VaB3FKkDrZ6RB1e7J96kHDiA3Hw0Kj2CyvGIxliiEYTDtN_ulHPMxh0NPtRHFYQwqsg==\bia.gov]8;;\
├── [12] ]8;id=129827;https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQH-gf6qX-4EZOApTVtyHtLK7t7rpv6mfTsPnof60RulnO3n0bc_e9EZgOy5-89U1digMnmNqm72gBd0iydJ9WnqEjqhZy_AKEgg52Wmcuj4icv9y4kK0aO0

⚠ Answer may differ

Expected: Swinomish Indian Tribal Community, Upper Skagit Indian Tribe

## Summary

In this notebook, you learned:

1. How to create and configure a `KnowledgeGroundedAgent`
2. The system instructions that guide agent behavior
3. How to use the agent for single-turn Q&A
4. How to explore the DeepSearchQA evaluation dataset

**Next**: In the next notebook, we'll explore multi-turn conversations and run systematic evaluations.

In [12]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]03_multi_turn.ipynb[/bold] to learn about multi-turn conversations and evaluation.",
        title="🎉 Done",
        border_style="green",
    )
)

╭──────────────────────────────────────────────────── 🎉 Done ────────────────────────────────────────────────────╮
│ ✓ Notebook complete!                                                                                            │
│                                                                                                                 │
│ Next: Open 03_multi_turn.ipynb to learn about multi-turn conversations and evaluation.                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯